<h1>Capstone project comparing Hong Kong and Singapore -SG

In [26]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


<h2>Define Foursquare Credentials and Version

<h2>Geo Coordinates of Singapore 

In [28]:
address = 'Downtown, Singapore'

geolocator = Nominatim(user_agent="sg_explorer")
location = geolocator.geocode(address)
sglatitude = location.latitude
sglongitude = location.longitude
print('The geograpical coordinate of Downtown, Singapore are {}, {}.'.format(sglatitude, sglongitude))

The geograpical coordinate of Downtown, Singapore are 1.2791912, 103.8533448.


<H2> Map of Singapore

In [29]:
sg_map = folium.Map(location=[sglatitude, sglongitude], zoom_start=15)
sg_map
folium.features.CircleMarker(
    [sglatitude, sglongitude],
    radius=30,
    color='red',
    popup='Downtown',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(sg_map)
sg_map

<h2> Singapore Foursquare Analysis

In [31]:
results = requests.get(url).json()

In [32]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [ ]:
sgvenues = results['response']['groups'][0]['items']
    
sgnearby_venues = json_normalize(sgvenues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
sgnearby_venues =sgnearby_venues.loc[:, filtered_columns]

# filter the category for each row
sgnearby_venues['venue.categories'] = sgnearby_venues.apply(get_category_type, axis=1)

# clean columns
sgnearby_venues.columns = [col.split(".")[-1] for col in sgnearby_venues.columns]

sgnearby_venues.head(5)

,name,categories,lat,lng
0,Marina Bay Downtown Area (MBDA),Harbor / Marina,1.279042,103.854956
1,LeVeL 33 Craft-Brewery Restaurant & Lounge,Brewery,1.280396,103.854204
2,SOS,Pub,1.279996,103.852925
3,Pure Fitness,Gym,1.278631,103.851487
4,Din Tai Fung 鼎泰豐,Dumpling Restaurant,1.280548,103.853843


<h2>Ranking the Catagories of venues

In [ ]:
sgnearby_venues.groupby('categories').count().head(100)

,name,lat,lng
categories,,,
Art Gallery,1,1,1
Bakery,1,1,1
Bar,2,2,2
Brewery,1,1,1
Buffet,1,1,1
Building,1,1,1
Burger Joint,1,1,1
Café,4,4,4
Chinese Restaurant,7,7,7


In [ ]:
sggroup=sgnearby_venues.groupby('categories').count()

In [ ]:
sggroup.sort_values('name', ascending=False).head(5)

,name,lat,lng
categories,,,
Coffee Shop,11,11,11
Chinese Restaurant,7,7,7
Japanese Restaurant,6,6,6
Sandwich Place,5,5,5
Café,4,4,4


In [ ]:
# selecet restaurarnt from categories 
# mark coffee shops on the map 
# analysis of Starbucks' presence in Central, Hong Kong and Downtown Singapore

In [ ]:
sggroup2=sggroup.filter(like='Restaurant', axis=0)  
sggroup2.sort_values('name', ascending=False)
sggroup2.drop(['lat','lng'], axis=1, inplace=True)
sggroup2.sort_values('name', ascending=False)
sggroup2.rename(columns={'name': 'counts'}, inplace=True)
sggroup2.sort_values('counts', ascending=False)

<h2>Locating Starbucks around the city centre

In [ ]:
search_query = 'Starbucks'
radius = 500
print(search_query + ' .... OK!')

In [ ]:
url2 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, sglatitude, sglongitude, VERSION, search_query, radius, LIMIT)
url2

In [ ]:
results = requests.get(url2).json()
results

In [ ]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

In [ ]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

In [ ]:
sgstarbucks_map = folium.Map(location=[sglatitude, sglongitude], zoom_start=13) # generate map centred around the Conrad Hotel

for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(sgstarbucks_map)

sgstarbucks_map

In [ ]:
sg_starbucks=dataframe_filtered[['name','categories','lat','lng']]
sg_starbucks

In [ ]:
dataframe_filtered.columns

<h2>Plotting Non-Starbucks around the city centre

In [ ]:
sg_coffee_shop=sgnearby_venues.loc[sgnearby_venues['categories'] == 'Coffee Shop']
sg_coffee_shop
sgcoffee_nonsb=sg_coffee_shop[sg_coffee_shop.name != 'Starbucks']
sgcoffee_nonsb

In [ ]:
# create map of Central using latitude and longitude values
map_sg = folium.Map(location=[sglatitude, sglongitude], zoom_start=16)

# add markers to map
for lat, lng, label in zip(sgcoffee_nonsb['lat'], sgcoffee_nonsb['lng'], sgcoffee_nonsb['name']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=0.7,
        parse_html=False).add_to(map_sg)  

#sgcoffee_map = folium.Map(location=[sglatitude, sglongitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
#folium.features.CircleMarker(
    #[sglatitude, sglongitude],
   # radius=10,
    #color='red',
   # popup='Downtown',
   # fill = True,
   # fill_color = 'red',
   # fill_opacity = 0.6
#).add_to(sgcoffee_map)

# add the Italian restaurants as blue circle markers

for lat, lng, label in zip(sg_starbucks['lat'], sg_starbucks['lng'], sg_starbucks['name']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7,
        parse_html=False).add_to(map_sg)  
    
map_sg

In [ ]:
from branca.element import Template, MacroElement

template = """
{% macro html(this, kwargs) %}

<!doctype html>
<html lang="en">
<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <title>jQuery UI Draggable - Default functionality</title>
  <link rel="stylesheet" href="//code.jquery.com/ui/1.12.1/themes/base/jquery-ui.css">

  <script src="https://code.jquery.com/jquery-1.12.4.js"></script>
  <script src="https://code.jquery.com/ui/1.12.1/jquery-ui.js"></script>
  
  <script>
  $( function() {
    $( "#maplegend" ).draggable({
                    start: function (event, ui) {
                        $(this).css({
                            right: "auto",
                            top: "auto",
                            bottom: "auto"
                        });
                    }
                });
});

  </script>
</head>
<body>

 
<div id='maplegend' class='maplegend' 
    style='position: absolute; z-index:9999; border:2px solid grey; background-color:rgba(255, 255, 255, 0.8);
     border-radius:6px; padding: 10px; font-size:14px; right: 20px; bottom: 20px;'>
     
<div class='legend-title'>Coffee Shops</div>
<div class='legend-scale'>
  <ul class='legend-labels'>
    <li><span style='background:blue;opacity:0.7;'></span>Starbucks</li>
    <li><span style='background:red;opacity:0.7;'></span>Non-Starbucks</li>

  </ul>
</div>
</div>
 
</body>
</html>

<style type='text/css'>
  .maplegend .legend-title {
    text-align: left;
    margin-bottom: 5px;
    font-weight: bold;
    font-size: 90%;
    }
  .maplegend .legend-scale ul {
    margin: 0;
    margin-bottom: 5px;
    padding: 0;
    float: left;
    list-style: none;
    }
  .maplegend .legend-scale ul li {
    font-size: 80%;
    list-style: none;
    margin-left: 0;
    line-height: 18px;
    margin-bottom: 2px;
    }
  .maplegend ul.legend-labels li span {
    display: block;
    float: left;
    height: 16px;
    width: 30px;
    margin-right: 5px;
    margin-left: 0;
    border: 1px solid #999;
    }
  .maplegend .legend-source {
    font-size: 80%;
    color: #777;
    clear: both;
    }
  .maplegend a {
    color: #777;
    }
</style>
{% endmacro %}"""

macro = MacroElement()
macro._template = Template(template)

map_sg.get_root().add_child(macro)